<a href="https://colab.research.google.com/github/KevinJayne2023/Agent_Coding_System_LangChain/blob/main/Agent_coding_app_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip -q install --upgrade langchain langchain-openai langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.8 MB/s eta 0:00:00


In [2]:
# API key and global settings
import os, getpass

from google.colab import userdata
userdata.get('OPENAI_API_KEY')

if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

MODEL = "gpt-4o-mini"
TEMPERATURE = 0.2
MAX_REVIEW_PASSES = 3
RUN_TIMEOUT_SECS = 15


Enter your OpenAI API key: ··········


In [3]:
# Define LangChain agents and a review loop LangGraph

from __future__ import annotations
import subprocess, tempfile, re
from typing import Optional, TypedDict

# LangChain core + OpenAI chat model
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage

# LangGraph
from langgraph.graph import StateGraph, END

# ---------- Helpers ----------
def llm():
    return ChatOpenAI(model=MODEL, temperature=TEMPERATURE)

def extract_python_block(text: str) -> str:
    """
    Extract the first ```python ... ``` code block. If none, try any fenced block.
    If still none, return the raw string.
    """
    m = re.search(r"```python(.*?)(```|$)", text, flags=re.S|re.I)
    if m:
        return m.group(1).strip()
    m = re.search(r"```(.*?)(```|$)", text, flags=re.S)
    if m:
        return m.group(1).strip()
    return text.strip()

def run_code_and_capture(code_str: str):
    """Run code in a temp file, capture stdout/stderr/returncode."""
    with tempfile.NamedTemporaryFile("w", suffix=".py", delete=False) as tmp:
        tmp.write(code_str)
        path = tmp.name
    try:
        proc = subprocess.run(
            ["python", path],
            capture_output=True,
            text=True,
            timeout=RUN_TIMEOUT_SECS,
        )
        return dict(stdout=proc.stdout, stderr=proc.stderr, ok=(proc.returncode == 0))
    except subprocess.TimeoutExpired as e:
        return dict(stdout="", stderr=f"Timeout after {RUN_TIMEOUT_SECS}s: {e}", ok=False)

# ---------- State for LangGraph ----------
class BuildState(TypedDict, total=False):
    app_idea: str
    plan: Optional[str]
    draft_code: Optional[str]
    code: Optional[str]
    run_stdout: Optional[str]
    run_stderr: Optional[str]
    success: bool
    passes: int

# ---------- Agent 1: Architect (LangChain) ----------
architect_system = """You are Agent 1: Architect.
Turn a short app idea or coding problem into a concise, implementable plan.

Constraints:
- Prefer a single-file Python solution unless multiple files are essential.
- Specify main components, functions/classes, inputs/outputs.
- Add simple acceptance checks (what "done" looks like).
- No code, just a plan.

Output sections:
1) Overview
2) Steps (numbered)
3) Minimal API of functions/classes
4) Acceptance checks (bullets)
"""
architect_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", architect_system),
        ("human", "App/Problem:\n{app_idea}\n\nProduce the plan."),
    ]
)
architect_chain = architect_prompt | llm() | StrOutputParser()

def architect_node(state: BuildState) -> BuildState:
    plan = architect_chain.invoke({"app_idea": state["app_idea"]})
    return {"plan": plan}

# ---------- Agent 2: Coder (LangChain) ----------
coder_system = """You are Agent 2: Coder.
Implement the Architect's plan as working Python in ONE file.

Rules:
- Output ONLY a single fenced ```python code block``` with the complete file.
- Include a simple `if __name__ == "__main__":` demo/CLI when feasible.
- Use only Python standard library unless absolutely necessary.
"""
coder_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", coder_system),
        ("human",
         "Here is the plan from the Architect. Implement it now as one Python file.\n\n"
         "{plan}\n\n"
         "Return ONLY a single ```python code block``` with the full file."
        ),
    ]
)
coder_chain = (coder_prompt | llm() | StrOutputParser())

def coder_node(state: BuildState) -> BuildState:
    raw = coder_chain.invoke({"plan": state["plan"]})
    code = extract_python_block(raw)
    return {"draft_code": code, "code": code}

# ---------- Agent 3: Reviewer (LangChain + tool execution) ----------
reviewer_system = """You are Agent 3: Reviewer.
You run the code, observe errors, and produce corrected code if needed.

Process:
1) If the code runs without error, return it unchanged.
2) If it fails, analyze the traceback and return a FIXED full-file version.

Rules:
- Always output ONLY a single fenced ```python code block``` with the complete corrected file.
- Keep design faithful to the Architect plan; improve robustness if trivial.
"""
reviewer_fix_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", reviewer_system),
        ("human",
         "The code failed with this traceback:\n\n"
         "```text\n{traceback}\n```\n\n"
         "Here is the full current code to fix:\n\n"
         "```python\n{code}\n```\n\n"
         "Return ONLY the full corrected file in a single ```python block."
        ),
    ]
)
reviewer_fix_chain = reviewer_fix_prompt | llm() | StrOutputParser()

def run_node(state: BuildState) -> BuildState:
    result = run_code_and_capture(state["code"])
    return {
        "run_stdout": result["stdout"],
        "run_stderr": result["stderr"],
        "success": result["ok"],
    }

def reviewer_node(state: BuildState) -> BuildState:
    # If already successful, no change
    if state.get("success"):
        return {}
    # Ask LLM to fix using traceback
    fixed_raw = reviewer_fix_chain.invoke({
        "traceback": state.get("run_stderr", "").strip()[:6000],
        "code": state["code"],
    })
    fixed_code = extract_python_block(fixed_raw)
    # increment passes
    new_passes = int(state.get("passes", 0)) + 1
    return {"code": fixed_code, "passes": new_passes}


In [4]:
# ---------- Build the LangGraph ----------
graph = StateGraph(BuildState)

graph.add_node("architect", architect_node)
graph.add_node("coder", coder_node)
graph.add_node("run", run_node)
graph.add_node("reviewer", reviewer_node)

graph.set_entry_point("architect")
graph.add_edge("architect", "coder")
graph.add_edge("coder", "run")

def route_after_run(state: BuildState):
    if state.get("success"):
        return "end"
    if int(state.get("passes", 0)) >= MAX_REVIEW_PASSES:
        return "end"
    return "reviewer"

graph.add_conditional_edges("run", route_after_run, {"reviewer": "reviewer", "end": END})
graph.add_edge("reviewer", "run")

app = graph.compile()

def build_with_team(app_idea: str) -> BuildState:
    initial: BuildState = {
        "app_idea": app_idea,
        "success": False,
        "passes": 0,
    }
    final_state = app.invoke(initial)
    return final_state

In [5]:
# Provide your app idea or coding problem, then run
APP_IDEA = """
Build a CLI that converts a CSV to JSON with selectable columns and basic validation.
"""

state = build_with_team(APP_IDEA)

print("\n=== Agent 1: Architect Plan ===\n")
print(state.get("plan", "").strip())

print("\n\n=== Agent 2: Coder Draft (first pass) ===\n")
print(state.get("draft_code", "").strip())

print("\n\n=== Agent 3: Reviewer (final code) ===\n")
print(state.get("code", "").strip())

print("\n\n=== Run Output (stdout) ===\n")
print((state.get("run_stdout") or "(no output)").strip())

print("\n=== Run Output (stderr) ===\n")
print((state.get("run_stderr") or "(no errors)").strip())

print(f"\nSuccess: {state.get('success')}  |  Review passes used: {state.get('passes')}")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [6]:
# Install Gradio
!pip -q install --upgrade gradio


In [8]:
# Launch a Gradio UI for the 3-agent team
import gradio as gr

# A small wrapper so you can tweak settings from the UI without editing prior cells
def run_pipeline(app_idea: str,
                 model: str = "gpt-4o-mini",
                 temperature: float = 0.2,
                 max_review_passes: int = 3,
                 timeout_secs: int = 15):

    # Use globals from earlier cells, but temporarily override them for this run
    global MODEL, TEMPERATURE, MAX_REVIEW_PASSES, RUN_TIMEOUT_SECS

    # Stash old values
    old_vals = (MODEL, TEMPERATURE, MAX_REVIEW_PASSES, RUN_TIMEOUT_SECS)

    try:
        MODEL = model
        TEMPERATURE = float(temperature)
        MAX_REVIEW_PASSES = int(max_review_passes)
        RUN_TIMEOUT_SECS = int(timeout_secs)

        state = build_with_team(app_idea)

        plan = (state.get("plan") or "").strip()
        draft_code = (state.get("draft_code") or "").strip()
        final_code = (state.get("code") or "").strip()
        stdout = (state.get("run_stdout") or "(no output)").strip()
        stderr = (state.get("run_stderr") or "(no errors)").strip()
        meta = {
            "success": bool(state.get("success")),
            "review_passes_used": int(state.get("passes", 0))
        }

        return plan, draft_code, final_code, stdout, stderr, meta

    finally:
        # Restore old values for any subsequent calls
        MODEL, TEMPERATURE, MAX_REVIEW_PASSES, RUN_TIMEOUT_SECS = old_vals


default_models = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1-mini",
]

with gr.Blocks(title="Agent Coding Team (LangChain + LangGraph)") as demo:
    gr.Markdown(
        """
        # 🧑‍💻 3-Agent Coding Team — LangChain + LangGraph
        **Agents:** Architect → Coder → Reviewer (runs & fixes code)
        Paste an app idea or coding problem, tune settings, and click **Run Agents**.
        """
    )

    with gr.Row():
        app_idea = gr.Textbox(
            label="App idea / coding problem",
            placeholder="e.g., Build a CLI that converts a CSV to JSON with selectable columns and basic validation.",
            lines=6,
        )

    with gr.Accordion("Advanced settings", open=False):
        model = gr.Dropdown(default_models, value="gpt-4o-mini", label="Model")
        temperature = gr.Slider(0.0, 1.0, value=0.2, step=0.05, label="Temperature")
        max_passes = gr.Slider(1, 6, value=3, step=1, label="Max review passes")
        timeout = gr.Slider(5, 60, value=15, step=1, label="Run timeout (seconds)")

    run_btn = gr.Button("🚀 Run Agents", variant="primary")

    gr.Markdown("### 📋 Architect Plan")
    out_plan = gr.Markdown()

    gr.Markdown("### 🧱 Coder Draft (first pass)")
    out_draft = gr.Code(language="python")

    gr.Markdown("### ✅ Reviewer Final Code")
    out_final = gr.Code(language="python")

    with gr.Row():
        out_stdout = gr.Textbox(label="Run Output (stdout)", lines=8)
        out_stderr = gr.Textbox(label="Run Output (stderr)", lines=8)

    out_meta = gr.JSON(label="Meta")

    run_btn.click(
        fn=run_pipeline,
        inputs=[app_idea, model, temperature, max_passes, timeout],
        outputs=[out_plan, out_draft, out_final, out_stdout, out_stderr, out_meta],
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6359df40d4e9e118b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
